In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# 학습 데이터 읽기
train_file_path = '../input/train.csv'  # 학습 데이터 파일 경로
train_data = pd.read_csv(train_file_path)

# 결측값 처리
train_data.fillna('WT', inplace=True)

# 테스트 데이터 읽기
test_file_path = '../input/test.csv'  # 테스트 데이터 파일 경로
test_data = pd.read_csv(test_file_path)

# 결측값 처리
test_data.fillna('WT', inplace=True)

# 테스트 데이터에 SUBCLASS 열 추가 (NaN으로 설정)
test_data['SUBCLASS'] = None

# 학습 데이터와 테스트 데이터 결합
combined_data = pd.concat([train_data, test_data], ignore_index=True)

# 특징과 타겟 분리
X = combined_data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
y = combined_data['SUBCLASS']




In [3]:
X

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8742,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
8743,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
8744,WT,WT,WT,WT,WT,L217I,P221P P251P,R5M,G606D,I248Nfs,...,S2049Vfs S1909Vfs,L232R,WT,WT,L305L,WT,N252I N251I,G679V,WT,WT
8745,WT,WT,WT,WT,WT,WT,WT,WT,R1517H,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [4]:
combined_data

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8742,TEST_2541,None,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
8743,TEST_2542,None,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
8744,TEST_2543,None,WT,WT,WT,WT,WT,L217I,P221P P251P,R5M,...,S2049Vfs S1909Vfs,L232R,WT,WT,L305L,WT,N252I N251I,G679V,WT,WT
8745,TEST_2544,None,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [5]:
# 특징 인코딩: 'WT'는 0, 그 외는 1로 변환
X_encoded = X.apply(lambda col: col.map(lambda x: 0 if x == 'WT' else 1))

# 레이블 인코딩 (학습 데이터에만 적용)
le = LabelEncoder()
y_encoded = le.fit_transform(y.dropna())  # NaN을 제외하고 인코딩

# 학습 데이터와 테스트 데이터 분리
y_encoded_full = pd.Series([None] * len(combined_data), index=combined_data.index)  # 초기화
y_encoded_full.iloc[:len(y_encoded)] = y_encoded  # 학습 데이터에 대해서만 값 설정


In [6]:

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_encoded[:len(y_encoded)], y_encoded, test_size=0.2, random_state=42)



In [7]:
X_train

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
286,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1820,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4384,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4822,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5226,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_val

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
5368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2463,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2505,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5418,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1315,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2346,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3751,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
724,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

# 모델 구성
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정
])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# 모델 학습
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=16, callbacks=[early_stopping])

# 모델 평가

# 예측 수행 (테스트 데이터)
X_test_encoded = X_encoded[len(y_encoded):]  # 테스트 데이터 부분
predictions = model.predict(X_test_encoded)

# 예측 결과 변환 (가장 높은 확률의 클래스를 선택)
predicted_classes = predictions.argmax(axis=1)

# 레이블 인코딩된 클래스를 원래 클래스 이름으로 변환
predicted_labels = le.inverse_transform(predicted_classes)

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('predictions3.csv', index=False)

print("Predictions saved to 'predictions.csv'.") 

#이 코드가 그나마 정확도가 0.246으로 가장 높았긴한데, 여전히 너무 낮아. 더 좋은 방법이 있을까?

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

# 학습 데이터 읽기
train_file_path = 'train_sample.csv'  # 학습 데이터 파일 경로
train_data = pd.read_csv(train_file_path)

# 결측값 처리
train_data.fillna('WT', inplace=True)

# 테스트 데이터 읽기
test_file_path = '/mnt/data/test.csv'  # 테스트 데이터 파일 경로
test_data = pd.read_csv(test_file_path)

# 결측값 처리
test_data.fillna('WT', inplace=True)

# 테스트 데이터에 SUBCLASS 열 추가 (NaN으로 설정)
test_data['SUBCLASS'] = None

# 학습 데이터와 테스트 데이터 결합
combined_data = pd.concat([train_data, test_data], ignore_index=True)

# 특징과 타겟 분리
X = combined_data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
y = combined_data['SUBCLASS']

# 특징 인코딩: 'WT'는 0, 그 외는 1로 변환
X_encoded = X.apply(lambda col: col.map(lambda x: 0 if x == 'WT' else 1))

# 레이블 인코딩 (학습 데이터에만 적용)
le = LabelEncoder()
y_encoded = le.fit_transform(y.dropna())  # NaN을 제외하고 인코딩

# 학습 데이터와 테스트 데이터 분리
y_encoded_full = pd.Series([None] * len(combined_data), index=combined_data.index)  # 초기화
y_encoded_full.iloc[:len(y_encoded)] = y_encoded  # 학습 데이터에 대해서만 값 설정

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_encoded[:len(y_encoded)], y_encoded, test_size=0.2, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 모델 구성
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정
])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), epochs=100, batch_size=16, callbacks=[early_stopping])

# 테스트 데이터 예측
X_test_encoded = X_encoded[len(y_encoded):]  # 테스트 데이터 부분
X_test_scaled = scaler.transform(X_test_encoded)  # 정규화 적용

predictions = model.predict(X_test_scaled)

# 예측 결과 변환 (가장 높은 확률의 클래스를 선택)
predicted_classes = predictions.argmax(axis=1)

# 레이블 인코딩된 클래스를 원래 클래스 이름으로 변환
predicted_labels = le.inverse_transform(predicted_classes)

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'Predicted_SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('predictions.csv', index=False)

print("Predictions saved to 'predictions.csv'.")
